In [2]:
import itertools
import io
import os
import cv2

import pandas as pd
import numpy as np

random_seed = 2
np.random.seed(random_seed)

from tqdm import tqdm
from random import shuffle

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


TRAIN_DIR = 'C:/Users/Kukus/Desktop/Kaggle/Dogs vs Cats/train/'
TEST_DIR = 'C:/Users/Kukus/Desktop/Kaggle/Dogs vs Cats/test/'
IMG_SIZE = 150
TOTAL_PIXELS = IMG_SIZE * IMG_SIZE
train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_images_dogs_cats = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

Using TensorFlow backend.


In [3]:
def label_img(img):
    label = img.split('.')[-3]
    if label == 'cat': return 1
    elif label == 'dog': return 0

In [4]:
#def create_train_data():
#    training_data = list()
#    for img in tqdm(os.listdir(TRAIN_DIR)):
#        label = label_img(img)
#        path = os.path.join(TRAIN_DIR, img)
#        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
#        training_data.append([np.array(img), np.array(label)])
#    shuffle(training_data)
#    np.save('train_data.npy', training_data)
#    return training_data
    

In [5]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in tqdm(list_of_images):
        img = cv2.resize(cv2.imread(image), (IMG_SIZE,IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        x.append(img)
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

In [6]:
def process_test_data():
    testing_data = list()
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

In [7]:
#train_data = prepare_data()
X, Y = prepare_data(train_images_dogs_cats)
#train_data = create_train_data()
# if you already have train data:
# train_data ? np.load

100%|██████████| 25000/25000 [01:38<00:00, 254.54it/s]


In [8]:
def show_img(idx):
    img = read_image(train_data[idx])
    plt.figure(figsize=(10,5))
    plt.imshow(img)
    plt.show()

In [9]:
# Consider Normalizing the data
#train_data = np.array(train_data)
#tt = train_data[0][0] / train_data[0][0].max(axis=0)
#norm_train_data = train_data / train_data.max(axis=0)
#norm_train_data

In [10]:
#X = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # Pixels
#X = np.array([i[0] for i in train_data])  # Pixels
#Y = [i[1] for i in train_data]  # Labels
#Y = [i[1][0] for i in train_data]  # Labels

In [11]:
# Split the train and validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=random_seed)

learning_rate = 0.01
epochs = 10
batch_size = 16
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)

In [13]:
# CNN Model

model = Sequential()

model.add(Convolution2D(32, 3, strides=3, padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3), activation='relu'))
#model.add(Convolution2D(32, 3, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, 3, strides=3, padding='same', activation='relu'))
#model.add(Convolution2D(64, 3, 3, padding='same', activation='relu'))
#model.add(Convolution2D(64, 3, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, strides=3, padding='same', activation='relu'))
#model.add(Convolution2D(128, 3, 3, padding='same', activation='relu'))
#model.add(Convolution2D(128, 3, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Convolution2D(256, 3, 3, padding='same', activation='relu'))
#model.add(Convolution2D(256, 3, 3, padding='same', activation='relu'))
#   model.add(Convolution2D(256, 3, 3, padding='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#   model.add(Convolution2D(256, 3, 3, padding='same', activation='relu'))
#   model.add(Convolution2D(256, 3, 3, padding='same', activation='relu'))
#   model.add(Convolution2D(256, 3, 3, padding='same', activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [14]:
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [15]:
model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)               

In [16]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [17]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [19]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [20]:
# Fit the model
#history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
#                              epochs = epochs, validation_data = (X_val,Y_val),
#                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
#                              , callbacks=[learning_rate_reduction])

history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    verbose = 1,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/30
1250/1250 [==============================] - 130s 104ms/step - loss: 0.6581 - accuracy: 0.5947 - val_loss: 0.5809 - val_accuracy: 0.6554
Epoch 2/30
1250/1250 [==============================] - 131s 105ms/step - loss: 0.5904 - accuracy: 0.6894 - val_loss: 0.4654 - val_accuracy: 0.7041
Epoch 3/30
1250/1250 [==============================] - 127s 102ms/step - loss: 0.5536 - accuracy: 0.7226 - val_loss: 0.6506 - val_accuracy: 0.7301
Epoch 4/30
1250/1250 [==============================] - 126s 101ms/step - loss: 0.5350 - accuracy: 0.7359 - val_loss: 0.5300 - val_accuracy: 0.7378
Epoch 5/30
1250/1250 [==============================] - 127s 102ms/step - loss: 0.5173 - accuracy: 0.7482 - val_loss: 0.4596 - val_accuracy: 0.7568
Epoch 6/30
1250/1250 [==============================] - 127s 102ms/step - loss: 0.5038 - accuracy: 0.7581 - val_loss: 0.6307 - val_accuracy: 0.7685
Epoch 7/30
1250/1250 [==============================] - 124s 99ms/step - loss: 0.4974 - accuracy: 0.7642 - val_l

In [21]:
model.save_weights('model_wieghts.h5')
model.save('model_keras.h5')

In [22]:
X_test, Y_test = prepare_data(test_images_dogs_cats)

100%|██████████| 12500/12500 [00:41<00:00, 298.95it/s]


In [23]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [24]:
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

782/782 [==============================] - 74s 94ms/step


In [25]:
counter = range(1, len(test_images_dogs_cats) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("solution.csv", index = False)